from https://github.com/brilee/python_uct

In [1]:
import collections
import numpy as np
import math

class UCTNode(): #the node object, it corresponds to a molecular state.
    def __init__(self, mol_state, move, parent=None): #a given node needs a molecular state
        self.mol_state = mol_state #associated State() object which is a molecular structure
        self.move = move # if a node was made with a move, that move index is linked here
        self.is_expanded = False # nodes start unexpanded
        self.parent = parent  # Optional[UCTNode] (I think if it's not the root it should have a parent?)
        self.children = {}  # Dict[move, UCTNode] starts with no children
        self.child_priors = np.zeros([362], dtype=np.float32) # an array of probabilities, indicating the current preference for each followup move
        self.child_number_visits = np.zeros([362], dtype=np.float32) # an array of the number of visits of each child
        self.children_total_values = np.zeros([362], dtype=np.float32) # an array of the total values of each of the children
        
    """Now that each node no longer knows about its own statistics, 
    we create aliases for a node’s statistics by using property getters and setters. 
    These allow us to transparently proxy these properties to the relevant entry in the parents’ child arrays."""
        
    @property
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value

    @property
    def total_value(self):
        return self.parent.children_total_values[self.move]

    @total_value.setter # self.move indexes the child to itself in the parent's children_total_values array. 
    def total_value(self, value):
        self.parent.children_total_values[self.move] = value

    def child_Q(self): # calculate Quality for the child arrays
        return self.children_total_values / (1 + self.child_number_visits)

    def child_U(self): # calculate Upper confidence bound for the child arrays
        return math.sqrt(self.number_visits) * (self.child_priors / (1 + self.child_number_visits))

    def best_child(self): #quickly finds the index of the child that has the highest aggregate of Q and U scores
        return np.argmax(self.child_Q() + self.child_U()) 

    def select_leaf(self):
        current = self
        while current.is_expanded: # if not expanded immediately return self
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current

    def expand(self, child_priors):
        self.is_expanded = True
        self.child_priors = child_priors

    def maybe_add_child(self, move):
        if move not in self.children:
            self.children[move] = UCTNode(
                self.mol_state.react(move), move, parent=self) # make a child with a reaction
        return self.children[move]

    def backup(self, value_estimate: float): #NEED to check 
        current = self
        while current.parent is not None:
            current.number_visits += 1
            current.total_value += value_estimate# used to (value_estimate * self.mol_state.to_play) # inverts based on turn, should try to remove
            current = current.parent #MIGHT not be right

class DummyNode(object): # makes a node without any child values or parent
    def __init__(self): 
        self.parent = None
        self.children_total_values = collections.defaultdict(float) 
        self.child_number_visits = collections.defaultdict(float)

def UCT_search(mol_state, num_reads):
    root = UCTNode(mol_state, move=None, parent=DummyNode()) # initializes the search starting with the given mol_state
# the parent is a dummy node because there shouldn't be a parent for the root molecule
    for _ in range(num_reads):
        leaf = root.select_leaf()
        child_priors, value_estimate = NeuralNet.evaluate(leaf.mol_state)
        leaf.expand(child_priors)
        leaf.backup(value_estimate)
    return np.argmax(root.child_number_visits)

class NeuralNet(): #returns an array of probabilities for followup moves and a value estimation of the current node
    @classmethod
    def evaluate(self, mol_state):
        return np.random.random([362]), np.random.random()

class State(): # NEED to make sure move indexes map to reactions
    def __init__(self, mol=None): #mol should be a molecule
        self.mol = mol
        self.name = str('foo') #SMILES or lookup method

    def react(self, move):
        return State(self.mol*move + np.random.random()) # eventually build and return a new structure

In [2]:
num_reads = 10000
import time
tick = time.time()
UCT_search(State(3), num_reads)
tock = time.time()
print("Took %s sec to run %s times" % (tock - tick, num_reads))
#import resource
#print("Consumed %sB memory" % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

Took 1.168428897857666 sec to run 10000 times
